In [2]:
pip install faker

  Using cached faker-37.1.0-py3-none-any.whl (1.9 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
# -------------------------------------------
# Interrelated HR Data Generation Script (Modified)
# -------------------------------------------
# This script generates 8 logically interconnected datasets:
# 1. Departments
# 2. Locations
# 3. Leave Types
# 4. Job Roles with Role Levels
# 5. Employees
# 6. Attendance Logs (with surplus/deficit logic)
# 7. Leave Records (reduced durations)
# 8. Projects (random names)
# -------------------------------------------

import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta, time

fake = Faker("en_IN")

# ----------------------
# 1. Departments Table
# ----------------------
department_names = [
    "Application development", "Data", "Business development", "Experienced Design",
    "Finance", "L&D", "Marketing", "People Operation", "Program Management",
    "Quality Engineer", "Talent Acquisition", "Test Engineering and Automation"
]
departments_df = pd.DataFrame({
    "department_id": list(range(1, len(department_names) + 1)),
    "department_name": department_names
})
departments_df.to_csv("departments.csv", index=False)

# ----------------------
# 2. Locations Table
# ----------------------
location_names = ["Bangalore", "Hyderabad"]
locations_df = pd.DataFrame({
    "location_id": [1, 2],
    "location_name": location_names
})
locations_df.to_csv("locations.csv", index=False)

# ----------------------
# 3. Leave Types Table
# ----------------------
leave_types = [
    "Earned Leave", "Sick Leave", "Family Caregiver Leave", "Comp Off Grant",
    "Bereavement Leave", "Loss of Pay (LOP)"
]
descriptions = [
    "Accrued monthly, carry forward up to 30 days.",
    "Used for illness, needs medical certificate beyond 2 days.",
    "Leave to care for sick family member.",
    "Compensatory leave granted to employees for working on official holidays or weekly offs",
    "Leave for death of immediate family.",
    "Unpaid leave when no paid leave is available."
]
leave_types_df = pd.DataFrame({
    "leave_type_id": list(range(1, len(leave_types) + 1)),
    "leave_type": leave_types,
    "description": descriptions
})
leave_types_df.to_csv("leave_types.csv", index=False)

# ----------------------
# 4. Job Roles Table
# ----------------------
designation_names = [
    "Associate Director - HR", "Associate Director - L&D", "Associate Director - Programs", "Associate Project Manager",
    "Business Development Executive", "Co -Founder CEO", "Co-Founder & COO", "CTO", "Data Analyst I", "Data Analyst II",
    "Data Analyst III", "Data Engineering Trainees", "DevOps Engineer III", "Director - Corporate Finance",
    "Director - Data Engineering", "Director - Sales", "Director of Technology", "Graphic Designer",
    "Graduate Engineering Intern", "HR Executive", "Junior Energy and Sustainability Officer", "Intern",
    "L&D Executive", "Lead-Talent Partner", "Manager - Marketing", "Marketing Executive", "Member of Technical Staff II",
    "Member of Technical Staff III", "Office Administrator", "Principal Engineer", "Project Manager", "Quality Engineer I",
    "Quality Engineer II", "Quality Engineer III", "Senior Business Analyst", "Senior Director Sales",
    "Senior Manager - Finance", "Senior Manager (Data Engineering Operations)", "Senior Principal Engineer",
    "Senior Product Manager", "Senior Talent Partner", "Senior UI/UX Designer", "Software Development Engineer I",
    "Software Development Engineer II", "Software Development Engineer III", "Software Development Engineer in Test I",
    "Software Development Engineer in Test II", "Software Development Engineer in Test III",
    "Software Development Engineer Trainees", "Software Engineer II", "Talent Partner", "Trainee Data Analyst",
    "Training Data Analyst", "UI/UX Designer", "Vice President - Data Engineering"
]

job_roles_df = pd.DataFrame({
    "designation_id": list(range(1, len(designation_names) + 1)),
    "designation_name": designation_names
})
job_roles_df.to_csv("job_roles.csv", index=False)

# ----------------------
NUM_EMPLOYEES = 448
NUM_INTERNS = 66
TOTAL_EMPLOYEES = NUM_EMPLOYEES + NUM_INTERNS  

employees = []
intern_counter = 1
fulltime_counter = 1
employment_types = ["Intern"] * NUM_INTERNS + ["Full-Time"] * NUM_EMPLOYEES
random.shuffle(employment_types)

def random_dob(min_year, max_year):
    start = datetime(min_year, 1, 1)
    end = datetime(max_year, 12, 31)
    return fake.date_between(start_date=start, end_date=end)

def random_join_date():
    return fake.date_between(start_date=datetime(2014, 1, 1), end_date=datetime(2024, 12, 31))

for i in range(TOTAL_EMPLOYEES):
    emp_type = employment_types[i]
    
    if emp_type == "Intern":
        employee_id = f"NLI-{intern_counter:03d}"
        dob = random_dob(2000, 2004)
        intern_counter += 1
    else:
        employee_id = f"NL-{fulltime_counter:03d}"
        dob = random_dob(1975, 2003)
        fulltime_counter += 1

    first_name = fake.first_name()
    last_name = fake.last_name()
    mail_id = f"{first_name.lower()}{last_name.lower()}@nineleaps.com"

    employees.append({
        "employee_id": employee_id,
        "first_name": first_name,
        "last_name": last_name,
        "full_name": f"{first_name} {last_name}",
        "mail_id": mail_id,
        "gender": random.choice(["Male", "Female"]),
        "dob": dob,
        "employment_type": emp_type,
        "department_id": random.randint(1, len(department_names)),
        "join_date": random_join_date(),
        "location_id": random.choice([1, 2])
    })

employees_df = pd.DataFrame(employees)
print(f"Total generated: {len(employees_df)}")  # Should print 448
employees_df.to_csv("employees.csv", index=False)

# ---------
# 6. Attendance Logs (With Clea Surplus/Deficit Formatting)
# ----------------------
attendance_data = []
for _, emp in employees_df.iterrows():
    join_date = pd.to_datetime(emp['join_date'])
    for day in pd.date_range(start=max(join_date, datetime(2024, 3, 1)), end=datetime(2025, 3, 31)):
        if day.weekday() < 5 and random.random() > 0.1:
            is_half_day = random.random() < 0.1
            if is_half_day:
                check_in_hour = random.randint(13, 14)
                duration_minutes = random.randint(180, 240)
            else:
                check_in_hour = random.randint(8, 10)
                duration_minutes = random.randint(510, 570)  # Around 9 hrs

            check_in = datetime.combine(day, time(hour=check_in_hour, minute=random.choice([0, 15, 30, 45])))
            check_out = check_in + timedelta(minutes=duration_minutes)
            working_hours = f"{duration_minutes // 60}hr {duration_minutes % 60}min"

            def format_time(minutes):
                hours = minutes // 60
                mins = minutes % 60
                if hours == 0:
                    return f"{mins}min"
                elif mins == 0:
                    return f"{hours}hr"
                else:
                    return f"{hours}hr {mins}min"

            if duration_minutes < 540:
                surplus = None
                deficit = format_time(540 - duration_minutes)
            else:
                deficit = None
                surplus = format_time(duration_minutes - 540)

            attendance_data.append({
                "employee_id": emp['employee_id'],
                "date": day.date(),
                "check_in": check_in.time(),
                "check_out": check_out.time(),
                "working_hours": working_hours,
                "surplus": surplus,
                "deficit": deficit,
                "is_half_day": is_half_day
            })

attendance_df = pd.DataFrame(attendance_data)
attendance_df.to_csv("attendance_logs.csv", index=False)


# ----------------------
# 7. Leave Records (Reduced Durations)
# ----------------------
leave_data = []
leave_type_dict = {1: "Earned Leave", 2: "Sick Leave", 3: "Family Caregiver Leave", 4: "Bereavement Leave", 5: "Loss of Pay (LOP)"}
for _, emp in employees_df.iterrows():
    join = pd.to_datetime(emp['join_date'])
    if join > datetime(2025, 1, 1):
        continue

    for leave_type_id, leave_type in leave_type_dict.items():
        if leave_type == "Sick Leave" and random.random() < 0.4:
            start = join + timedelta(days=random.randint(30, 240))
            leave_data.append({
                "employee_id": emp['employee_id'],
                "leave_type_id": leave_type_id,
                "start_date": start.date(),
                "end_date": (start + timedelta(days=3)).date(),
                "duration": 4,
                "reason": "Medical - Flu"
            })

        elif leave_type == "Earned Leave":
            for _ in range(random.randint(1, 3)):
                days = random.randint(2, 4)
                start = join + timedelta(days=random.randint(30, 300))
                leave_data.append({
                    "employee_id": emp['employee_id'],
                    "leave_type_id": leave_type_id,
                    "start_date": start.date(),
                    "end_date": (start + timedelta(days=days - 1)).date(),
                    "duration": days,
                    "reason": "Vacation"
                })

        elif leave_type == "Bereavement Leave" and random.random() < 0.1:
            days = random.randint(2, 4)
            start = join + timedelta(days=random.randint(60, 300))
            leave_data.append({
                "employee_id": emp['employee_id'],
                "leave_type_id": leave_type_id,
                "start_date": start.date(),
                "end_date": (start + timedelta(days=days - 1)).date(),
                "duration": days,
                "reason": "Family bereavement"
            })

        elif leave_type == "Family Caregiver Leave" and random.random() < 0.15:
            days = random.randint(2, 6)
            start = join + timedelta(days=random.randint(90, 330))
            leave_data.append({
                "employee_id": emp['employee_id'],
                "leave_type_id": leave_type_id,
                "start_date": start.date(),
                "end_date": (start + timedelta(days=days - 1)).date(),
                "duration": days,
                "reason": "Care for family"
            })

        elif leave_type == "Loss of Pay (LOP)" and random.random() < 0.1:
            days = random.randint(1, 2)
            start = join + timedelta(days=random.randint(20, 280))
            leave_data.append({
                "employee_id": emp['employee_id'],
                "leave_type_id": leave_type_id,
                "start_date": start.date(),
                "end_date": (start + timedelta(days=days - 1)).date(),
                "duration": days,
                "reason": "LOP"
            })

leave_df = pd.DataFrame(leave_data)
leave_df.to_csv("leave_records.csv", index=False)

# ----------------------
# 8. Projects Table (with Realistic Unique Names, No Suffixes)
# ----------------------
project_names_pool = [
    "Orion", "Nova", "Athena", "Pulse", "Nimbus", "Fusion", "Spectra", "Helix", "Zenith", "Vertex",
    "Apollo", "Horizon", "Equinox", "Eclipse", "Velocity", "Ignite", "Quantum", "Catalyst", "Sierra", "Omega"
]

# Sample 30 unique project names from the pool
unique_project_names = random.sample(project_names_pool * 2, 30)

project_data = []
for pid in range(1, 31):
    project_name = unique_project_names[pid - 1]  # Use clean unique name
    tech_stack = random.choice(["Python", "Java", "Node.js", "React", "Data Engineering", "DevOps"])
    lead = employees_df[employees_df['employment_type'] == 'Full-Time'].sample(1).iloc[0]['employee_id']
    assigned = employees_df.sample(random.randint(5, 12))['employee_id'].tolist()
    for emp_id in assigned:
        project_data.append({
            "project_id": pid,
            "project_name": project_name,
            "tech_stack": tech_stack,
            "employee_id": emp_id,
            "lead_id": lead
        })

projects_df = pd.DataFrame(project_data)
projects_df.to_csv("projects.csv", index=False)

#
# 9. Productivity Table
# ----------------------
productivity_data = []
for _, emp in employees_df.iterrows():
    employee_id = emp['employee_id']
    
    # Randomly assign a project to the employee
    assigned_projects = projects_df[projects_df['employee_id'] == employee_id]['project_id'].unique()
    
    # If no project assigned, skip this employee
    if not assigned_projects.any():
        continue
    
    project_id = random.choice(assigned_projects)
    
    # Generate random productivity data for each employee (let's assume we want daily records for 1 month)
    for day in pd.date_range(start=datetime(2024, 3, 1), end=datetime(2025, 3, 31), freq='MS'):
        # Skip weekends (Saturday and Sunday)
        if day.weekday() >= 5:
            continue
        
        tasks_assigned = random.randint(3, 10)  # Random tasks assigned between 3 and 10
        tasks_completed = random.randint(2, tasks_assigned)  # Tasks completed are between 0 and tasks assigned
        task_quality_score = round(random.uniform(0.0, 5.0), 2)  # Random task quality score between 0 and 5
        
        # Calculate productivity score
        if tasks_assigned > 0:
            productivity_score = (tasks_completed / tasks_assigned) * task_quality_score
        else:
            productivity_score = 0
        
        productivity_score = round(productivity_score, 2)

        # Assign meaningful remarks based on productivity_score
        if productivity_score >= 4.0:
            remarks = "Outstanding Contributor"
        elif productivity_score >= 3.0:
            remarks = "Consistently Effective"
        elif productivity_score >= 2.0:
            remarks = "Needs Closer Monitoring"
        elif productivity_score >= 1.0:
            remarks = "Performance Improvement Plan"
        else:
            remarks = "Critical Underperformance"
        
        productivity_data.append({
            "employee_id": employee_id,
            "project_id": project_id,
            "assigned_date": day.date(),
            "tasks_assigned": tasks_assigned,
            "tasks_completed": tasks_completed,
            "task_quality_score": task_quality_score,
            "productivity_score": productivity_score,
            "remarks": remarks
        })

# Convert to DataFrame
productivity_df = pd.DataFrame(productivity_data)

# Save to CSV
productivity_df.to_csv("productivity.csv", index=False)

print("Productivity data generated successfully.")




Total generated: 514
Productivity data generated successfully.
